In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Process X

In [3]:
import os
import pandas as pd
import numpy as np

stock_price_path = 'gdrive/MyDrive/Columbia/BDA_PJ/stock_price.csv'
sentiments_path = 'gdrive/MyDrive/Columbia/BDA_PJ/sentiments'
sentiment_fnames = os.listdir(sentiments_path)
print(sentiment_fnames[:5])

['mu.csv', 'ilmn.csv', 'foxa.csv', 'gild.csv', 'celg.csv']


In [20]:
def get_one_stock_sent(stock_index, sentiments_path, weighted=False):
  sent_fname = '{}.csv'.format(stock_index.lower())
  sent_df = pd.read_csv(os.path.join(sentiments_path, sent_fname), index_col=0).sort_values(by=['Date', 'Hour'])
  sent_df.rename(columns={'Hour': 'Time'}, inplace=True)
  sent_df['Hour'] = sent_df['Time'].apply(lambda x:x.split(':')[0])
  sim_sent_df = sent_df[['Date', 'Hour', 'compound', 'Followers']].dropna()

  # compute average sentiment score per hour
  sim_sent_df = sim_sent_df.groupby(['Date', 'Hour']).agg('mean').reset_index()
  sim_sent_df['Hour'] = sim_sent_df['Hour'].astype(int)
  # get X for this stock sent
  sent_X_df = {'Date':[]}
  for i in range(24):
    sent_X_df[str(i)] = []

  for date, group in sim_sent_df.groupby('Date'):
    sent_X_df['Date'].append(date)
    available_hours = set(group['Hour'])
    for i in range(24):
      if i in available_hours:
        sent_X_df[str(i)].append(group.loc[group['Hour']==i,'compound'].values[0])
      else:
        sent_X_df[str(i)].append(0)
  sent_X_df = pd.DataFrame.from_dict(sent_X_df)
  # sent_X_df['avg'] = sent_X_df.apply(lambda x: np.mean(x[[str(i) for i in range(24)]]), axis=1)
  if not weighted:
    sent_X_df['avg'] = sent_X_df['Date'].apply( lambda date: np.mean(sim_sent_df.loc[sim_sent_df['Date']==date, 'compound']) )
  else:
    sent_X_df['avg'] = sent_X_df['Date'].apply( lambda date: np.average(sim_sent_df.loc[sim_sent_df['Date']==date, 'compound'],
                                                                        weights = sim_sent_df.loc[sim_sent_df['Date']==date, 'Followers']) )
  sent_X_df.insert(0, 'index', stock_index)

  return sent_X_df

In [21]:
weighted = True
X_df = []
for fname in sentiment_fnames:
  stock_index = fname.split('.')[0].upper()
  cur_X_df = get_one_stock_sent(stock_index, sentiments_path, weighted)
  X_df.append(cur_X_df)
X_df = pd.concat(X_df)

In [9]:
X_df = X_df.sort_values(by=['index', 'Date'])
X_df = X_df.reset_index(drop=True)
print(X_df.shape)
X_df.head(10)

(7740, 27)


,index,Date,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,avg
0,AAL,2016-03-10,0.000,0.0000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000,0.00000,0.000000,0.0000,0.000000,-0.089100,-0.056575,0.500300,0.000000,0.033717,0.202300,0.296000,0.000000,0.073960,0.000000,0.00000,0.000000,-0.014734
1,AAL,2016-03-11,0.000,0.0000,0.470900,0.0000,0.0000,-0.340000,0.000000,0.000,0.00000,0.050575,0.0000,0.000000,0.084571,0.000000,0.077200,0.556950,0.488390,0.278700,0.503050,0.000000,0.000000,0.000000,0.00000,0.000000,0.261009
2,AAL,2016-03-12,0.000,0.0000,0.000000,0.0000,0.0000,0.000000,0.458800,0.000,0.00000,0.000000,0.0000,0.000000,0.296000,-0.340000,0.000000,0.000000,0.077200,0.000000,0.148000,0.000000,0.000000,0.000000,0.00000,0.000000,0.086331
3,AAL,2016-03-13,0.000,0.0772,0.000000,0.0000,0.0772,0.786500,0.000000,0.000,0.00000,-0.077200,0.0000,0.000000,0.000000,0.000000,0.000000,0.585900,0.585900,0.585900,0.000000,0.364867,0.000000,0.000000,-0.36120,-0.077200,0.270537
4,AAL,2016-03-14,0.000,0.0000,0.077200,0.0000,0.0000,0.000000,0.000000,0.000,0.14800,0.229400,0.0000,0.188700,0.000000,0.000000,0.809100,0.502975,0.202300,0.478520,0.000000,0.000000,0.000000,0.000000,0.14680,0.098780,0.251693
5,AAL,2016-03-15,0.000,0.0000,0.000000,-0.4019,0.0000,0.000000,0.098667,-0.064,-0.34035,-0.022860,0.2368,-0.033487,0.329267,0.377240,0.325400,0.296000,-0.135925,0.338900,0.000000,0.529700,0.000000,0.000000,0.00000,0.000000,0.046808
6,AAL,2016-03-16,0.000,0.0000,0.365867,0.5488,0.0000,0.296000,0.296000,0.000,0.00000,0.059238,0.0000,0.000000,0.148000,0.000000,0.162700,0.000000,0.268667,0.531250,0.296000,0.000000,0.112775,0.000000,0.00000,0.000000,0.499147
7,AAL,2016-03-17,0.000,0.0000,0.000000,0.0000,0.6037,0.098667,0.000000,0.000,0.03415,0.000000,0.0000,0.000000,0.249880,0.156420,0.000000,0.014900,0.108460,0.371500,-0.200950,0.000000,-0.401900,0.296000,0.33525,0.000000,0.065325
8,AAL,2016-03-18,0.000,0.0000,-0.401900,0.0000,0.0000,0.000000,0.000000,0.000,0.00000,0.067433,0.0000,0.014714,0.000000,0.207483,0.155350,-0.008600,0.356444,0.370214,0.224086,0.188543,0.215125,0.202383,-0.05630,0.145886,0.144965
9,AAL,2016-03-19,0.436,0.1366,0.000000,0.4019,0.0000,0.000000,0.000000,0.000,0.00000,0.000000,0.0000,0.000000,-0.140550,-0.064000,0.113333,0.000000,0.146800,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.150821


In [22]:
X_df.loc[X_df['avg'].isna(),:]

,index,Date,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,avg


In [23]:
if weighted:
  X_df.to_csv('gdrive/MyDrive/Columbia/BDA_PJ/weighted_stock_sents_by_date.csv')
else:
  X_df.to_csv('gdrive/MyDrive/Columbia/BDA_PJ/stock_sents_by_date.csv')

## Process Y

In [24]:
import datetime

stock_price_df = pd.read_csv('gdrive/MyDrive/Columbia/BDA_PJ/stock_price.csv')
stock_price_df['label'] = (stock_price_df['Close'] > stock_price_df['Open'])
stock_price_df['Date'] = pd.to_datetime(stock_price_df['Date'], format='%Y-%m-%d')

stock_price_df.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume,index,label
0,2016-03-10,42.110001,42.110001,40.910000,41.630001,39.892078,5814200.0,AAL,False
1,2016-03-11,41.889999,42.500000,41.700001,42.470001,40.697018,5701500.0,AAL,True
2,2016-03-14,42.310001,42.700001,42.119999,42.590000,40.811993,4568000.0,AAL,True
3,2016-03-15,42.299999,42.490002,41.980000,42.110001,40.352039,4445400.0,AAL,False
4,2016-03-16,41.810001,42.430000,41.590000,42.310001,40.543678,4792900.0,AAL,True
5,2016-03-17,42.310001,42.310001,41.820000,42.099998,40.342449,5960600.0,AAL,False
6,2016-03-18,42.310001,43.529999,41.750000,43.439999,41.626507,13746000.0,AAL,True
7,2016-03-21,43.320000,43.779999,43.279999,43.470001,41.655258,4756400.0,AAL,True
8,2016-03-22,42.000000,43.020000,41.840000,42.759998,40.974892,6721600.0,AAL,True
9,2016-03-23,42.799999,42.860001,41.580002,41.919998,40.169964,9583400.0,AAL,False


In [25]:
# compute y
day_duration = datetime.timedelta(1) 
stock_price_df['prev_date'] = (stock_price_df['Date'] - day_duration)

# compute the feature of previous up/down rate
stock_price_df.insert(0, 'prev_label', None)
stock_price_df.insert(0, 'prev_rate', None)

for i, row in stock_price_df.iterrows():
  prev_date = row['prev_date']
  if not stock_price_df.loc[(stock_price_df['index']==row['index'])&(stock_price_df['Date']==prev_date),:].empty:
    prev_info = stock_price_df.loc[(stock_price_df['index']==row['index'])&(stock_price_df['Date']==prev_date),:]
    stock_price_df.loc[i, 'prev_label'] = (prev_info['Close'] > prev_info['Open']).values[0]
    stock_price_df.loc[i, 'prev_rate'] = ((prev_info['Close'] - prev_info['Open'])/prev_info['Open']).values[0]

In [26]:
Y_df = stock_price_df[['index', 'prev_date', 'prev_label', 'prev_rate', 'label']].dropna()
Y_df.head(10)

,index,prev_date,prev_label,prev_rate,label
1,AAL,2016-03-10,False,-0.0113987,True
3,AAL,2016-03-14,True,0.00661779,False
4,AAL,2016-03-15,False,-0.00449169,True
5,AAL,2016-03-16,True,0.0119589,False
6,AAL,2016-03-17,False,-0.00496343,True
8,AAL,2016-03-21,True,0.00346264,True
9,AAL,2016-03-22,True,0.0180952,False
10,AAL,2016-03-23,False,-0.0205608,False
12,AAL,2016-03-28,False,-0.00219513,True
13,AAL,2016-03-29,True,0.0110024,False


## Merge as dataset

In [27]:
if weighted:
  X_df = pd.read_csv('gdrive/MyDrive/Columbia/BDA_PJ/weighted_stock_sents_by_date.csv', index_col=0)
else:
  X_df = pd.read_csv('gdrive/MyDrive/Columbia/BDA_PJ/stock_sents_by_date.csv', index_col=0)

X_df['Date'] = pd.to_datetime(X_df['Date'], format='%Y-%m-%d')
X_df.head()

,index,Date,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,avg
0,MU,2016-03-31,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.000000,0.000000,0.0000,0.000000,-0.161847,0.066121,0.116211,0.140607,0.053459,-0.072261,0.082465,-0.045258,0.122358,-0.113983,0.16650,0.205533,0.037124
1,MU,2016-04-01,0.372814,0.263350,0.130280,-0.023633,0.667550,0.0000,-0.1468,0.000,0.000000,0.233300,0.0000,0.197175,0.300756,0.083370,0.215310,0.039956,-0.022227,0.086350,0.129750,0.010592,0.185525,0.042989,0.03558,0.119175,0.232725
2,MU,2016-04-02,-0.318200,0.254667,0.034767,0.000000,0.067433,0.4588,0.0386,-0.125,0.000000,0.499500,0.1480,0.000000,0.000000,0.000000,0.000000,0.283150,-0.041833,-0.381800,0.159000,0.000000,0.000000,0.335250,0.00000,0.440400,0.093959
3,MU,2016-04-03,0.000000,0.000000,0.466050,-0.249750,0.226300,0.0000,0.0000,0.000,0.159100,0.000000,0.3753,0.000000,0.000000,0.000000,0.000000,0.474687,0.000000,0.000000,0.612400,0.502800,0.000000,0.227900,0.09095,0.099700,0.217567
4,MU,2016-04-04,0.315700,0.000000,0.612400,0.516950,0.065350,0.0000,0.4939,0.000,-0.051033,0.164383,-0.3182,0.000000,0.007756,0.000000,0.151183,-0.004064,-0.119491,-0.088200,0.000000,0.238863,-0.064060,0.000000,0.27320,0.254050,0.179038


In [28]:
dataset = X_df.merge(Y_df, left_on=['index','Date'], right_on=['index','prev_date'])
dataset.drop(columns = 'prev_date', inplace=True)
print(dataset.shape)
dataset.head(10)

(3566, 30)


,index,Date,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,avg,prev_label,prev_rate,label
0,MU,2016-03-31,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0000,0.000000,0.000000,0.000000,0.000000,-0.161847,0.066121,0.116211,0.140607,0.053459,-0.072261,0.082465,-0.045258,0.122358,-0.113983,0.166500,0.205533,0.037124,True,0.0155189,True
1,MU,2016-04-04,0.315700,0.00000,0.612400,0.516950,0.065350,0.000000,0.49390,0.0000,-0.051033,0.164383,-0.318200,0.000000,0.007756,0.000000,0.151183,-0.004064,-0.119491,-0.088200,0.000000,0.238863,-0.064060,0.000000,0.273200,0.254050,0.179038,False,-0.0235082,False
2,MU,2016-04-05,0.000000,0.00000,0.202300,-0.044000,0.140500,0.000000,0.13660,0.1480,0.000000,0.276967,0.305067,0.074150,0.255320,0.041667,0.094867,0.310431,0.110671,-0.170000,-0.056575,0.218333,0.098667,0.416700,0.000000,0.122667,0.161318,False,-0.0216369,True
3,MU,2016-04-06,0.293767,0.00000,0.170000,0.000000,0.000000,0.000000,-0.31245,0.0000,0.193750,-0.148000,-0.005940,0.000000,0.329850,0.000000,0.145486,0.287025,0.090300,-0.315520,0.000000,0.000000,0.000000,0.318200,0.000000,0.642750,-0.001381,True,0.00479848,True
4,MU,2016-04-07,0.035580,0.39310,0.282550,0.735100,0.105375,0.000000,0.00000,0.1779,0.000000,0.000000,0.000000,0.000000,-0.016000,0.210750,0.021444,0.140120,0.000000,0.017200,0.394000,0.034233,0.242220,0.098667,0.000000,0.000000,0.186738,True,0.00863725,True
5,MU,2016-04-11,0.000000,0.42460,0.000000,0.000000,0.000000,0.000000,0.00000,-0.5859,0.238967,0.050800,0.307900,0.055536,0.079900,0.167683,-0.065200,0.498760,0.047825,0.121933,0.316000,0.232589,-0.004476,0.169300,0.042743,0.141369,-0.072365,False,-0.027907,False
6,MU,2016-04-12,0.063200,0.44505,0.203381,0.181400,-0.648600,0.000000,-0.78450,0.0000,0.000000,0.375300,0.000000,-0.571900,-0.303150,0.000000,0.002033,0.026644,0.084040,0.144543,0.234394,0.055400,-0.042037,0.060347,0.123100,0.092136,-0.048226,False,-0.0323502,True
7,MU,2016-04-13,0.170867,0.04628,0.081018,-0.039891,0.134100,0.173242,0.40120,0.7269,0.000000,-0.087883,-0.127650,-0.303400,0.043260,0.244983,-0.072533,0.082700,-0.065550,0.068025,0.264000,0.000000,0.412000,0.027114,-0.432400,0.000000,0.041844,True,0.0563107,False
8,MU,2016-04-14,0.631450,-0.02580,0.256540,0.000000,0.000000,0.487325,0.17000,0.0000,0.158900,0.493900,0.261650,0.321520,0.624900,0.313344,-0.114487,0.131986,0.000000,-0.113725,0.093500,0.000000,0.020350,0.122480,0.401900,0.108233,0.224793,False,-0.0361446,False
9,MU,2016-04-18,0.150120,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0000,0.210750,0.322733,0.164633,0.012900,0.119000,0.000000,0.063691,-0.042433,0.012025,0.039500,0.075500,0.145380,0.000000,0.054760,0.074000,0.240800,0.068238,True,0.0244591,False


In [29]:
if weighted:
  dataset.to_csv('gdrive/MyDrive/Columbia/BDA_PJ/weighted_dataset.csv')
else:
  dataset.to_csv('gdrive/MyDrive/Columbia/BDA_PJ/dataset.csv')

## Get train & test

In [30]:
import numpy as np
from sklearn.model_selection import train_test_split

if weighted:
  dataset = pd.read_csv('gdrive/MyDrive/Columbia/BDA_PJ/weighted_dataset.csv', index_col=0)
else:
  dataset = pd.read_csv('gdrive/MyDrive/Columbia/BDA_PJ/dataset.csv', index_col=0)

train, test = train_test_split(dataset, test_size=0.2, random_state=0, 
                               stratify=dataset['index'])

print(train.shape, test.shape)
train.head(10)

(2852, 30) (714, 30)


,index,Date,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,avg,prev_label,prev_rate,label
2184,ADBE,2016-06-02,0.296000,0.000000,0.0000,0.0000,0.324300,0.0000,0.381800,0.0000,0.05160,0.00000,0.00000,0.0000,0.000000,0.440400,0.616450,0.000000,0.38535,0.38180,0.00000,0.6369,0.000000,0.440400,0.0000,-0.458500,0.289575,True,0.001900,False
3152,LRCX,2016-04-20,0.000000,0.000000,0.0000,0.0000,-0.147533,0.0000,0.000000,0.0000,0.00000,0.00000,0.00000,0.0000,0.000000,0.000000,-0.153100,0.000000,0.00000,0.00000,0.00000,0.4019,0.000000,0.096378,0.0000,-0.136600,0.021646,True,0.003764,False
41,ILMN,2016-03-28,0.000000,0.202300,0.0000,0.0000,0.318200,0.0000,0.000000,0.0000,0.00000,0.00000,0.00000,0.0000,0.000000,-0.064000,0.000000,0.000000,0.25000,0.00000,0.00000,0.0000,-0.223500,0.000000,0.0000,0.000000,0.017349,False,-0.028358,True
3095,DISCA,2016-03-29,0.000000,0.000000,0.0000,0.0000,0.000000,0.0000,0.381800,0.0000,0.00000,0.00000,0.00000,0.3400,0.000000,0.381800,0.279033,0.400300,0.00000,0.00000,0.00000,0.0000,0.000000,0.578800,0.0000,0.000000,0.364460,True,0.001774,False
536,TRIP,2016-04-21,0.000000,0.000000,0.0000,0.0000,0.000000,0.0000,0.000000,0.0000,0.27115,0.38850,0.00000,0.0000,0.000000,0.246950,0.351250,-0.085333,0.34000,0.00000,0.00000,0.0000,0.000000,0.000000,0.4246,0.000000,0.148458,False,-0.012993,False
920,INTU,2016-06-06,0.381800,0.000000,0.0000,0.0000,0.000000,0.0000,0.000000,0.3818,0.38180,0.09545,0.00000,0.0000,0.683300,0.266317,0.177900,0.000000,0.00000,0.00000,0.00000,-0.1280,0.000000,0.000000,0.0000,0.000000,-0.125588,True,0.007172,True
2240,EBAY,2016-04-06,0.218625,0.113333,-0.1366,0.0000,0.000000,0.0000,0.254533,0.0000,0.00000,0.17790,0.00000,0.4939,0.395120,0.187736,0.125829,0.239450,0.12284,0.13064,0.00000,0.0000,0.068423,0.124936,0.1749,0.432133,0.135003,True,0.040082,False
1897,WDC,2016-04-21,0.000000,0.000000,0.0000,0.1779,0.000000,-0.3612,0.663300,0.0000,0.20230,0.39415,0.00000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.0869,0.000000,0.000000,0.2263,0.000000,0.031564,False,-0.016338,True
332,TMUS,2016-05-12,0.220200,0.000000,0.4215,0.0000,0.273200,0.4215,-0.273200,0.5948,0.00000,0.00000,0.20406,0.0000,0.132120,0.275280,0.340000,0.029300,0.17020,0.00000,0.23835,0.0000,0.000000,0.636900,0.0000,0.159100,0.335364,False,-0.002694,True
517,NCLH,2016-06-07,0.000000,0.000000,0.0000,0.0000,0.000000,0.0000,0.000000,0.0000,0.00000,0.00000,0.00000,0.0000,0.254533,0.000000,0.202300,0.180600,0.00000,0.00000,0.00000,0.0000,0.648600,0.000000,0.4939,0.000000,0.252089,True,0.003278,False


In [31]:
if weighted:
  train.to_csv('gdrive/MyDrive/Columbia/BDA_PJ/weighted_train.csv')
  test.to_csv('gdrive/MyDrive/Columbia/BDA_PJ/weighted_test.csv')
else:
  train.to_csv('gdrive/MyDrive/Columbia/BDA_PJ/train.csv')
  test.to_csv('gdrive/MyDrive/Columbia/BDA_PJ/test.csv')